In [52]:
import pandas as pd

In [53]:
# Load the data

df = pd.read_csv('mass_doe_data.csv')
df.shape

(636, 2776)

In [54]:
# Remove columns with less than two unique values

df = df[df.columns[df.apply(lambda x: x.nunique() > 1, axis=0)]]
df.shape

(636, 1695)

In [55]:
# Remove columns with more than 90% missing values

df = df[df.columns[df.isna().sum() / len(df) < 0.90]]
df.shape

(636, 1502)

In [56]:
# Remove non-middle school and non-mathematics MCAS scores

a = df[[c for c in df.columns if 'MCAS' not in c]]

b = df[[c for c in df.columns 
        if 'MCAS' in c 
        and 'MATHEMATICS' in c 
        and ('ALL MIDDLE SCHOOL GRADES' in c 
             or 'GRADE 06' in c 
             or 'GRADE 07' in c 
             or 'GRADE 08' in c)]]
b = b.reindex(sorted(b.columns), axis=1)

df = pd.concat([a, b], axis=1)
df.shape

(636, 503)

In [57]:
demographics = df[[c for c in df.columns if 'MCAS' not in c]]
demographics.shape

(636, 323)